# Molecular Dynamics 4 

Ok, now that we have multiple atoms, lets have them interact with each other.  
  
So lets restart all of our previous turtle things to get window set up.

In [1]:
import turtle
import random
import math

When using random numbers, it is important to start a seed to insure that you actually are getting random numbers.

In [2]:
random.seed()

Now lets get our back drop set up.

In [3]:
window = turtle.Screen()
window.title('Molecular Dynamics 4')
window.clear()

Lets see how big our window is, which will tell us where the boundaries are.

In [4]:
# We will want to store these numbers in a variable
height = window.window_height()
width = window.window_width()

Lets check and see what we have here.

In [5]:
print height
print width

810
960


Now we have our window, lets make a turtles to draw our atoms. We are going to make a list of atoms and then populate it.

In [6]:
# Number of atoms we want
num_atoms = 5

# This initializes an empty list
atom_list = []

# Now use a loop to initialize a new atom
# and do it num_atoms times.
for i in range(num_atoms):
    atom_list.append(turtle.Turtle())

We should now have two atoms stored in atom_list. We can check various aspects of this new list using print.

In [7]:
print len(atom_list)

5


In [8]:
print atom_list

[<turtle.Turtle object at 0x105d88e10>, <turtle.Turtle object at 0x105a2b890>, <turtle.Turtle object at 0x105d88e50>, <turtle.Turtle object at 0x105d88f10>, <turtle.Turtle object at 0x105d88fd0>]


Everything looks good. Now lets draw our atoms as spheres with a fixed radius and random color and place them at random locations.

In [9]:
# Variables to hold things we want to be constant
atom_radius = 20

# Scaling factor here is so we don't get our 
# initial positions stuck on the edge.
scaling_factor = 0.8

# We need to loop over each atom.
for i in range(num_atoms):
   
    # Draw the atom in the proper shape
    atom_list[i].shape('circle')
    atom_list[i].shapesize(atom_radius/10.0)
    atom_list[i].color((random.random(),random.random(),random.random()))
    atom_list[i].penup()
    atom_list[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, random.uniform(-1,1)*height/2.0 * scaling_factor)
  
    # Turtles can be very slow. This is a semi-fix to tell
    # turtles not to update the screen with every change,
    # but rather wait till a set of updates are done and 
    # then update the screen.
    atom_list[i].tracer(0,0)
    turtle.update()

Alright, now lets make a vector with random velocities for each atom. Our velocity vector will have one line per atom, and each line will have a pair of floating point numbers that represent (velocity_x, velocity_y).

In [10]:
# Max velocity we want
max_velocity = 20.0

# Initailize an empty list
velocity_list = []

# Now loop over the number of atoms
for i in range(num_atoms):
    velocity_list.append([random.uniform(-1,1)*max_velocity, random.uniform(-1,1)*max_velocity])

Lets check to make sure this worked.

In [11]:
print len(velocity_list)

5


In [12]:
print velocity_list

[[10.364002886288603, -6.609671206255494], [-14.02930102215075, 18.85218068701321], [-10.788579572083442, 17.11363607649031], [19.423244302814147, -2.6230701131016776], [-1.6187703788307983, 6.377075418182132]]


Now how would we access only 1 element? We use square brackets []!

In [13]:
print velocity_list[0]

[10.364002886288603, -6.609671206255494]


Now how would we access the x_velocity component here? We'd use another set of square brackets!

In [14]:
print velocity_list[0][0]

10.3640028863


Now we need to write a function that will handle the interaction of any two atoms.  
  
First we will need, at some point, to calculate distances between two atoms, so lets write a function that handles that.

In [15]:
# Lets name our function
def distance(atom1, atom2):
    
    # Inputs are the atoms, which contain
    # (x,y) coordinate pairs.
    # Distance is sqrt((x1-x2)^2 + (y1-y2)^2)
    
    # Lets get the coordinates of atom1
    (x1,y1) = atom1.pos()
    
    # Lets get the coordinates of atom2
    (x2,y2) = atom2.pos()

    # Lets calculate the distance
    d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    # And lets return our distance
    return d
    

Now lets write a function to determine if our atoms will collide, and will handle what to do when they collide.

In [16]:
# Lets name our function
def bounce_check(atom_list, v_list):
  
    # This will help us not double count
    i = 1

    # We need to check for every atom
    for atom1 in atom_list:
        
        # This will help us not double count
        j = i
        
        # And compute the distance to every other atom
        for atom2 in atom_list[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                (x1,y1) = atom1.pos()
                (x2,y2) = atom2.pos()
                
                # A bounce should occur: fix x_velocity
                # First, calculate v1.r
                v1R = v_list[i-1][0]*(x1-x2) + v_list[i-1][1]*(y1-y2)
                v2R = v_list[j][0]*(x1-x2) + v_list[j][1]*(y1-y2)
                
                # Update new velocities
                v_list[i-1][0] = v_list[i-1][0] - 2.0*v1R/d**2 * (x1-x2)
                v_list[i-1][1] = v_list[i-1][1] - 2.0*v1R/d**2 * (y1-y2)
                v_list[j][0] = v_list[j][0] - 2.0*v2R/d**2 * (x1-x2)
                v_list[j][1] = v_list[j][1] - 2.0*v2R/d**2 * (y1-y2)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1

In [17]:
# Lets name our function
def forces_check(atom_list, v_list, dt):

    # Our spring constant
    k = 0.05

    # This will help us not double count
    i = 1

    # We need to check for every atom
    for atom1 in atom_list:
        
        # This will help us not double count
        j = i
        
        # And compute the distance to every other atom
        for atom2 in atom_list[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                (x1,y1) = atom1.pos()
                (x2,y2) = atom2.pos()
                
                # A bounce should occur
                # Update new velocities
                v_list[i-1][0] = v_list[i-1][0] + dt/2.0*k*d*(x1-x2)
                v_list[i-1][1] = v_list[i-1][1] + dt/2.0*k*d*(y1-y2)
                v_list[j][0] = v_list[j][0] + dt/2.0*k*d*(x2-x1)
                v_list[j][1] = v_list[j][1] + dt/2.0*k*d*(y2-y1)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1

Now to start moving our atoms.

In [18]:
# The amount of time each iteration moves us forward
dt = 0.5

for i in range(1,1100):
    
    # Just a little string formatting to make output nicer
    #print "Iteration %d" % i
    
    Vx = 0.0
    Vy = 0.0
    for i in range(num_atoms):
        Vx += velocity_list[i][0] 
        Vy += velocity_list[i][1]
    print math.sqrt(Vx**2+Vy**2)
    
    # Check for bounces
    #bounce_check(atom_list, velocity_list) # Funny.
    forces_check(atom_list, velocity_list, dt)
    
    # We want to move each atom
    for j in range(num_atoms):
    
        # Get the current position of the atom
        (x,y) = atom_list[j].pos()
    
        #print "   Atom %d: (%3.2f, %3.2f)" % (j, x, y)
    
        # Check if moving left or right will put our atom beyond the wall
        if abs(x + dt * velocity_list[j][0]) >= width/2.0 - atom_radius:
        
            # We have moved too far right or left, so flip the x_vel
            velocity_list[j][0] = -velocity_list[j][0]           
    
        # Check if moving up or down will put our atom beyond the wall
        if abs(y + dt * velocity_list[j][1]) >= height/2.0 - atom_radius:
        
            # We have moved too far up or down, so flip the y_vel
            velocity_list[j][1] = -velocity_list[j][1]
        
        # We won't move out of the box, so update the new position
        atom_list[j].goto(x + dt*velocity_list[j][0], y + dt*velocity_list[j][1])
        
        # Tell turtles we are done updating and to redraw
        turtle.update()
    

33.279251571
33.279251571
33.279251571
33.279251571
33.279251571
33.279251571
33.279251571
33.279251571
3.53191944712
3.53191944712
35.513466974
35.513466974
35.513466974
35.513466974
35.513466974
35.513466974
35.513466974
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
BOUNCE FOUND: Atom 1 hit Atom 4
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
13.9634915095
43.7278489384
43.7278489384
43.7278489384
43.

It works!

Always clean up.

In [19]:
window.bye()